In [1]:
import gc
import os
import uproot
import pandas
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.callbacks import ModelCheckpoint, CSVLogger
from statsmodels.stats.proportion import proportion_confint

C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
central_tau_id_branches = ['againstElectronMVA6Raw', 'byCombinedIsolationDeltaBetaCorrRaw3Hits',
    'byIsolationMVArun2v1DBoldDMwLTraw', 'byIsolationMVArun2v1DBdR03oldDMwLTraw',
    'byIsolationMVArun2v1DBoldDMwLTraw2016', 'byIsolationMVArun2017v2DBoldDMwLTraw2017',
    'byIsolationMVArun2017v2DBoldDMdR0p3wLTraw2017', 'id_flags']
truth_branches = ['gen_match']
input_branches = ['pt', 'eta', 'mass', 'decayMode', 'chargedIsoPtSum', 'neutralIsoPtSum', 'neutralIsoPtSumWeight',
                  'photonPtSumOutsideSignalCone', 'puCorrPtSum',
                  'dxy', 'dxy_sig', 'dz', 'ip3d', 'ip3d_sig',
                  'hasSecondaryVertex', 'flightLength_r', 'flightLength_dEta', 'flightLength_dPhi',
                  'flightLength_sig', 'leadChargedHadrCand_pt', 'leadChargedHadrCand_dEta',
                  'leadChargedHadrCand_dPhi', 'leadChargedHadrCand_mass', 'pt_weighted_deta_strip',
                  'pt_weighted_dphi_strip', 'pt_weighted_dr_signal', 'pt_weighted_dr_iso',
                  'leadingTrackNormChi2', 'e_ratio', 'gj_angle_diff', 'n_photons', 'emFraction',
                  'has_gsf_track', 'inside_ecal_crack',
                  'gsf_ele_matched', 'gsf_ele_pt', 'gsf_ele_dEta', 'gsf_ele_dPhi', 'gsf_ele_mass', 'gsf_ele_Ee',
                  'gsf_ele_Egamma', 'gsf_ele_Pin', 'gsf_ele_Pout', 'gsf_ele_EtotOverPin', 'gsf_ele_Eecal',
                  'gsf_ele_dEta_SeedClusterTrackAtCalo', 'gsf_ele_dPhi_SeedClusterTrackAtCalo', 'gsf_ele_mvaIn_sigmaEtaEta',
                  'gsf_ele_mvaIn_hadEnergy',
                  'gsf_ele_mvaIn_deltaEta', 'gsf_ele_Chi2NormGSF', 'gsf_ele_GSFNumHits', 'gsf_ele_GSFTrackResol',
                  'gsf_ele_GSFTracklnPt', 'gsf_ele_Chi2NormKF', 'gsf_ele_KFNumHits',
                  'leadChargedCand_etaAtEcalEntrance', 'leadChargedCand_pt', 'leadChargedHadrCand_HoP',
                  'leadChargedHadrCand_EoP', 'tau_visMass_innerSigCone', 'n_matched_muons', 'muon_pt', 'muon_dEta', 'muon_dPhi',
                  'muon_n_matches_DT_1', 'muon_n_matches_DT_2', 'muon_n_matches_DT_3', 'muon_n_matches_DT_4',
                  'muon_n_matches_CSC_1', 'muon_n_matches_CSC_2', 'muon_n_matches_CSC_3', 'muon_n_matches_CSC_4',
                  'muon_n_hits_DT_2', 'muon_n_hits_DT_3', 'muon_n_hits_DT_4',
                  'muon_n_hits_CSC_2', 'muon_n_hits_CSC_3', 'muon_n_hits_CSC_4',
                  'muon_n_hits_RPC_2', 'muon_n_hits_RPC_3', 'muon_n_hits_RPC_4',
                  'muon_n_stations_with_matches_03', 'muon_n_stations_with_hits_23', 
                  'signalChargedHadrCands_sum_innerSigCone_pt', 'signalChargedHadrCands_sum_innerSigCone_dEta',
                  'signalChargedHadrCands_sum_innerSigCone_dPhi', 'signalChargedHadrCands_sum_innerSigCone_mass',
                  'signalChargedHadrCands_sum_outerSigCone_pt', 'signalChargedHadrCands_sum_outerSigCone_dEta',
                  'signalChargedHadrCands_sum_outerSigCone_dPhi', 'signalChargedHadrCands_sum_outerSigCone_mass',
                  'signalChargedHadrCands_nTotal_innerSigCone', 'signalChargedHadrCands_nTotal_outerSigCone',
                  'signalNeutrHadrCands_sum_innerSigCone_pt', 'signalNeutrHadrCands_sum_innerSigCone_dEta',
                  'signalNeutrHadrCands_sum_innerSigCone_dPhi', 'signalNeutrHadrCands_sum_innerSigCone_mass',
                  'signalNeutrHadrCands_sum_outerSigCone_pt', 'signalNeutrHadrCands_sum_outerSigCone_dEta',
                  'signalNeutrHadrCands_sum_outerSigCone_dPhi', 'signalNeutrHadrCands_sum_outerSigCone_mass',
                  'signalNeutrHadrCands_nTotal_innerSigCone', 'signalNeutrHadrCands_nTotal_outerSigCone',
                  'signalGammaCands_sum_innerSigCone_pt', 'signalGammaCands_sum_innerSigCone_dEta',
                  'signalGammaCands_sum_innerSigCone_dPhi', 'signalGammaCands_sum_innerSigCone_mass',
                  'signalGammaCands_sum_outerSigCone_pt', 'signalGammaCands_sum_outerSigCone_dEta',
                  'signalGammaCands_sum_outerSigCone_dPhi', 'signalGammaCands_sum_outerSigCone_mass',
                  'signalGammaCands_nTotal_innerSigCone', 'signalGammaCands_nTotal_outerSigCone',
                  'isolationChargedHadrCands_sum_pt', 'isolationChargedHadrCands_sum_dEta',
                  'isolationChargedHadrCands_sum_dPhi', 'isolationChargedHadrCands_sum_mass',
                  'isolationChargedHadrCands_nTotal',
                  'isolationNeutrHadrCands_sum_pt', 'isolationNeutrHadrCands_sum_dEta',
                  'isolationNeutrHadrCands_sum_dPhi', 'isolationNeutrHadrCands_sum_mass',
                  'isolationNeutrHadrCands_nTotal',
                  'isolationGammaCands_sum_pt', 'isolationGammaCands_sum_dEta',
                  'isolationGammaCands_sum_dPhi', 'isolationGammaCands_sum_mass',
                  'isolationGammaCands_nTotal',
                 ]
all_branches = truth_branches + input_branches + central_tau_id_branches
match_suffixes = [ 'e', 'mu', 'tau', 'jet' ]
gen_match_ex_branches = [ 'gen_{}'.format(suff) for suff in match_suffixes ]
e, mu, tau, jet = 0, 1, 2, 3
input_shape = (len(input_branches), )
n_outputs = len(gen_match_ex_branches)

class GenMatch:
    Electron = 1
    Muon = 2
    TauElectron = 3
    TauMuon = 4
    Tau = 5
    NoMatch = 6 

In [3]:
def ReadBrancesToDataFrame(file_name, tree_name, branches, nentries=None):
    with uproot.open(file_name) as file:
        tree = file[tree_name]
        df = tree.arrays(branches, entrystop=nentries, outputtype=pandas.DataFrame)
        df.columns = [ c.decode('utf-8') for c in df.columns ]
    return df

In [4]:
def ReadBranchesTo2DArray(file_name, tree_name, branches, dtype, chunk_size = 20, nentries=None):
    data = None
    n = 0
    branch_chunks = [ branches[pos:pos+chunk_size] for pos in range(0, len(branches), chunk_size) ]
    if nentries is None:
        with uproot.open(file_name) as file:
            tree = file[tree_name]
            nentries = tree.numentries
        gc.collect()
    data = np.empty([nentries, len(branches)], dtype=dtype)
    for chunk in branch_chunks:
        df = ReadBrancesToDataFrame(file_name, tree_name, chunk, nentries)
        for br in chunk:
            data[:, n] = df[br].astype(dtype)
            print("branch '{}' loaded. {}/{}".format(br, n + 1, len(branches)))
            n += 1
        del df
        gc.collect()
    return data

In [5]:
def VectorizeGenMatch(data, dtype):
    if data.shape[1] != 1:
        raise RuntimeError("Invalid input")
    v_data = np.zeros([data.shape[0], 4], dtype=dtype)
    v_data[:, e] = ((data[:, 0] == GenMatch.Electron) | (data[:, 0] == GenMatch.TauElectron)).astype(dtype)
    v_data[:, mu] = ((data[:, 0] == GenMatch.Muon) | (data[:, 0] == GenMatch.TauMuon)).astype(dtype)
    v_data[:, tau] = (data[:, 0] == GenMatch.Tau).astype(dtype)
    v_data[:, jet] = (data[:, 0] == GenMatch.NoMatch).astype(dtype)
    return v_data

In [6]:
input_file = '../../tuples/mixed_training/shuffled.root'

In [7]:
X = ReadBranchesTo2DArray(input_file, 'taus', input_branches, np.float32, chunk_size = 30)
X.shape

branch 'pt' loaded. 1/129
branch 'eta' loaded. 2/129
branch 'mass' loaded. 3/129
branch 'decayMode' loaded. 4/129
branch 'chargedIsoPtSum' loaded. 5/129
branch 'neutralIsoPtSum' loaded. 6/129
branch 'neutralIsoPtSumWeight' loaded. 7/129
branch 'photonPtSumOutsideSignalCone' loaded. 8/129
branch 'puCorrPtSum' loaded. 9/129
branch 'dxy' loaded. 10/129
branch 'dxy_sig' loaded. 11/129
branch 'dz' loaded. 12/129
branch 'ip3d' loaded. 13/129
branch 'ip3d_sig' loaded. 14/129
branch 'hasSecondaryVertex' loaded. 15/129
branch 'flightLength_r' loaded. 16/129
branch 'flightLength_dEta' loaded. 17/129
branch 'flightLength_dPhi' loaded. 18/129
branch 'flightLength_sig' loaded. 19/129
branch 'leadChargedHadrCand_pt' loaded. 20/129
branch 'leadChargedHadrCand_dEta' loaded. 21/129
branch 'leadChargedHadrCand_dPhi' loaded. 22/129
branch 'leadChargedHadrCand_mass' loaded. 23/129
branch 'pt_weighted_deta_strip' loaded. 24/129
branch 'pt_weighted_dphi_strip' loaded. 25/129
branch 'pt_weighted_dr_signal' l

(35000000, 129)

In [8]:
Y_raw = ReadBranchesTo2DArray(input_file, 'taus', truth_branches, int)
Y = VectorizeGenMatch(Y_raw, int)
Y.shape

branch 'gen_match' loaded. 1/1


(35000000, 4)

In [9]:
def tau_e_loss(target, output):
    sf = tf.convert_to_tensor(3, output.dtype.base_dtype)
    epsilon = tf.convert_to_tensor(1e-7, output.dtype.base_dtype)
    tau_vs_e = output[:, tau] / (output[:, tau] + output[:, e] + epsilon)
    loss_tau_e = target[:, tau] * tf.log(tau_vs_e) + target[:, e] * tf.log(1 - tau_vs_e)
    return -sf * loss_tau_e

def tau_mu_loss(target, output):
    sf = tf.convert_to_tensor(20, output.dtype.base_dtype)
    epsilon = tf.convert_to_tensor(1e-7, output.dtype.base_dtype)
    tau_vs_mu = output[:, tau] / (output[:, tau] + output[:, mu] + epsilon)
    tau_vs_mu = tf.clip_by_value(tau_vs_mu, epsilon, 1 - epsilon)
    loss_tau_mu = target[:, tau] * tf.log(tau_vs_mu) + target[:, mu] * tf.log(1 - tau_vs_mu)
    return -sf * loss_tau_mu

def tau_jet_loss(target, output):
    sf = tf.convert_to_tensor(1, output.dtype.base_dtype)
    epsilon = tf.convert_to_tensor(1e-7, output.dtype.base_dtype)
    tau_vs_jet = output[:, tau] / (output[:, tau] + output[:, jet] + epsilon)
    tau_vs_jet = tf.clip_by_value(tau_vs_jet, epsilon, 1 - epsilon)
    loss_tau_jet = target[:, tau] * tf.log(tau_vs_jet) + target[:, jet] * tf.log(1 - tau_vs_jet)
    return -sf * loss_tau_jet

def tau_crossentropy(target, output):
    return tau_e_loss(target, output) + tau_mu_loss(target, output) + tau_jet_loss(target, output)

In [10]:
def UpdateClassWeights(weight_df, tau_class_weight = 1):
    other_classes_weight = 1
    all_class_weight = tau_class_weight + other_classes_weight
    tau_weights = weight_df[weight_df['gen_tau'] == 1]['weight']
    other_weights = weight_df[weight_df['gen_tau'] != 1]['weight']
    n_taus = tau_weights.shape[0]
    w_taus = tau_weights.sum()
    n_other = other_weights.shape[0]
    w_other = other_weights.sum()
    sf = (n_taus + n_other) * tau_class_weight / (w_taus * all_class_weight)
    sf_other = sf * w_taus * other_classes_weight / (w_other * tau_class_weight)

    weight_df.loc[weight_df['gen_tau'] == 1, 'weight'] *= sf
    weight_df.loc[weight_df['gen_tau'] != 1, 'weight'] *= sf_other
    
    return sf, sf_other

In [11]:
def SetBinWeights(weight_df, bin_weights):
    n_bins = len(bin_weights)
    for pt_bin in range(0, n_bins):
        weight_df.loc[(weight_df['gen_tau'] == 1) & (weight_df['pt'] > pt_bins[pt_bin])
                      & (weight_df['pt'] <= pt_bins[pt_bin+1]), 'weight'] = bin_weights[pt_bin]


In [12]:
def UpdateWeights(model, weight_df, bin_weights, x, test_start, n_test, target_eff, boost=1, tau_class_weight = 1):
    pred = model.predict(x[test_start:test_start+n_test], batch_size = 250000, verbose=0)
    for n in range(len(match_suffixes)):
        br_suff = match_suffixes[n]
    br_loc = weight_df.columns.get_loc('tau_vs_jet')
    tau_vs_jet = pred[:, tau] / (pred[:, tau] + pred[:, jet] + 1e-9)
    weight_df.iloc[test_start:test_start+n_test, br_loc] = tau_vs_jet

    df_sel = weight_df[(weight_df['gen_tau'] == 1) & (weight_df['pt'] <= pt_bins[-1])
                       & (weight_df['orig_index'] >= test_start) & (weight_df['orig_index'] < test_start + n_test)]
    thr = np.percentile(df_sel.tau_vs_jet, (1-target_eff) * 100)
    print('Average threshold for eff={}: {}'.format(target_eff, thr))
    df_jet = weight_df[(weight_df['gen_jet'] == 1)
                       & (weight_df['orig_index'] >= test_start) & (weight_df['orig_index'] < test_start + n_test)]
    fpr = np.count_nonzero(df_jet['tau_vs_jet'] > thr) / df_jet.shape[0]
    print('tau_vs_jet fpr at threshold: ', fpr)

    n_bins = len(bin_weights)
    new_bin_weights = np.zeros(n_bins)
    eff = np.zeros(n_bins)
    eff_up = np.zeros(n_bins)
    eff_down = np.zeros(n_bins)
    for pt_bin in range(0, n_bins):
        df_pt = df_sel[(df_sel['pt'] > pt_bins[pt_bin]) & (df_sel['pt'] <= pt_bins[pt_bin+1])]

        n_tot = df_pt.shape[0]
        n_sel = np.count_nonzero(df_pt['tau_vs_jet'] > thr)
        eff[pt_bin] = n_sel / n_tot
        eff_down[pt_bin], eff_up[pt_bin] = proportion_confint(n_sel, n_tot, 1-0.682689, 'beta')
        if target_eff > eff_down[pt_bin] and target_eff < eff_up[pt_bin]:
            w = bin_weights[pt_bin]
        else:
            sf = target_eff / eff[pt_bin]
            if sf < 1:
                boost = 1 / boost
            w = sf * bin_weights[pt_bin] * boost
        new_bin_weights[pt_bin] = w

    SetBinWeights(weight_df, new_bin_weights)
    sf, sf_other = UpdateClassWeights(weight_df, tau_class_weight)     
    new_bin_weights = new_bin_weights * sf
    
    table_header_format = '{:<8}{:<8}{:<20}{:<15}{:<15}'
    table_format = '{:<8}{:<8}{:<20}{:<15.2f}{:<15.2f}'
    table_eff_format = '{:.1f} [{:.1f}, {:.1f}]'
    print(table_header_format.format("pt min", "pt max", "eff", "old weight", "new weight"))
    for pt_bin in range(0, n_bins):
        eff_str = table_eff_format.format(eff[pt_bin] * 100, eff_down[pt_bin] * 100, eff_up[pt_bin] * 100)
        print(table_format.format(pt_bins[pt_bin], pt_bins[pt_bin+1], eff_str,
                                  bin_weights[pt_bin], new_bin_weights[pt_bin]))

    return new_bin_weights

In [12]:
n_hidden_layers = 20
n_neurons = 1024
model_name = "{}L{}N".format(n_hidden_layers, n_neurons)
model = Sequential()
model.add(Dense(n_neurons, input_shape=input_shape, kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))
for n in range(n_hidden_layers):
    model.add(Dense(n_neurons, kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
model.add(Dense(n_outputs))
model.add(Activation("softmax"))

In [13]:
#model = load_model('10L1024N_0607_s1_loss.hdf5')
#model_name = '10L1024N'
opt = keras.optimizers.Adam(lr=1e-3)
#model.compile(loss="categorical_crossentropy", optimizer=opt, weighted_metrics=["accuracy"])
model.compile(loss=tau_crossentropy, optimizer=opt, weighted_metrics=["accuracy"], metrics=["accuracy"])

In [13]:
tau_class_weight = 0.4
pt_bins = np.array([20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200, 1000])
bin_weights = np.ones(len(pt_bins) - 1)
weight_df = pandas.DataFrame({ 'pt': X[:, 0], 'weight': np.ones(X.shape[0]) })
for n in range(len(match_suffixes)):
    br_suff = match_suffixes[n]
    weight_df['gen_'+br_suff] = pandas.Series(Y[:, n], index=weight_df.index)
weight_df['tau_vs_jet'] = pandas.Series(np.zeros(X.shape[0]), index=weight_df.index)
weight_df['orig_index'] = pandas.Series(weight_df.index.values, index=weight_df.index)
sf, sf_other = UpdateClassWeights(weight_df, tau_class_weight)
bin_weights *= sf
print(sf, sf_other)

1.0 1.0


In [15]:
train_name = '{}_0610_s1'.format(model_name)
n_train = int(3e7)
n_test = X.shape[0] - n_train
n_test_weight = int(1e6)
n_per_step = int(5e6)
epoch = 0
n_pass = 25

cb_acc = ModelCheckpoint("{}_acc.hdf5".format(train_name), monitor="val_acc", save_best_only=True,
                         save_weights_only=False, mode="max", verbose=1)
cb_w_acc = ModelCheckpoint("{}_w_acc.hdf5".format(train_name), monitor="val_weighted_acc", save_best_only=True,
                           save_weights_only=False, mode="max", verbose=1)
cb_loss = ModelCheckpoint("{}_loss.hdf5".format(train_name), monitor="val_loss", save_best_only=True,
                          save_weights_only=False, mode="min", verbose=1)
csv_log = CSVLogger("{}.log".format(train_name), append=False)

for pass_id in range(0, n_pass):
    print("Starting pass {}".format(pass_id))
    for pos in range(0, n_train, n_per_step):
        if epoch >= 5:
            val_data = (X[n_train:n_train+n_test, :], Y[n_train:n_train+n_test, :],
                                          weight_df.weight[n_train:n_train+n_test].values)
            callbacks = [cb_acc, cb_w_acc, cb_loss, csv_log]
        else:
            val_data = None
            callbacks = [csv_log]
        fit_hist = model.fit(X[pos:pos+n_per_step, :], Y[pos:pos+n_per_step, :], callbacks=callbacks,
                             validation_data=val_data, batch_size=10000, epochs=epoch+1, initial_epoch=epoch, verbose=1,
                             sample_weight=weight_df.weight[pos:pos+n_per_step].values)
        print("Estimating new weights...")
        #boost = 1 + 0.1 / (epoch + 1)
        bin_weights = UpdateWeights(model, weight_df, bin_weights, X, pos, n_test_weight, 0.6,
                                    boost=1, tau_class_weight=tau_class_weight)
        epoch += 1
model.save("{}_final.hdf5".format(train_name))

Starting pass 0
Epoch 1/1
5000000/5000000 [==============================] - 272s 54us/step - loss: 4.2987 - acc: 0.5402 - weighted_acc: 0.5402
Estimating new weights...
Average threshold for eff=0.6: 0.031723237782716754
tau_vs_jet fpr at threshold:  0.08478825717570308
pt min  pt max  eff                 old weight     new weight     
20      30      60.9 [60.8, 61.1]   1.00           0.98           
30      40      61.1 [61.0, 61.3]   1.00           0.98           
40      50      58.2 [57.9, 58.4]   1.00           1.03           
50      60      56.0 [55.7, 56.4]   1.00           1.07           
60      70      57.0 [56.5, 57.5]   1.00           1.05           
70      80      56.3 [55.7, 56.9]   1.00           1.06           
80      90      57.8 [57.1, 58.5]   1.00           1.04           
90      100     59.2 [58.3, 60.1]   1.00           1.00           
100     120     61.2 [60.3, 62.0]   1.00           0.98           
120     140     64.6 [63.4, 65.7]   1.00           0.93   


Epoch 00007: val_acc did not improve from 0.80489

Epoch 00007: val_weighted_acc did not improve from 0.80280

Epoch 00007: val_loss improved from 2.96685 to 2.80351, saving model to 20L1024N_0610_s1_loss.hdf5
Estimating new weights...
Average threshold for eff=0.6: 0.5624966859817505
tau_vs_jet fpr at threshold:  0.023279560345069304
pt min  pt max  eff                 old weight     new weight     
20      30      58.6 [58.5, 58.8]   1.29           1.31           
30      40      61.3 [61.2, 61.5]   0.92           0.89           
40      50      61.4 [61.1, 61.6]   0.81           0.79           
50      60      60.1 [59.8, 60.5]   0.77           0.76           
60      70      62.0 [61.6, 62.5]   0.63           0.60           
70      80      61.4 [60.8, 61.9]   0.53           0.52           
80      90      61.0 [60.2, 61.7]   0.47           0.46           
90      100     59.8 [58.9, 60.7]   0.44           0.44           
100     120     59.1 [58.3, 60.0]   0.41           0.42    

Average threshold for eff=0.6: 0.5184553503990174
tau_vs_jet fpr at threshold:  0.022799322940532857
pt min  pt max  eff                 old weight     new weight     
20      30      59.9 [59.7, 60.0]   1.37           1.37           
30      40      60.6 [60.4, 60.8]   0.83           0.82           
40      50      59.5 [59.3, 59.8]   0.74           0.74           
50      60      58.7 [58.3, 59.1]   0.73           0.75           
60      70      59.2 [58.7, 59.7]   0.55           0.56           
70      80      60.5 [59.9, 61.1]   0.45           0.45           
80      90      62.8 [62.1, 63.5]   0.39           0.37           
90      100     59.2 [58.3, 60.1]   0.38           0.38           
100     120     61.8 [61.0, 62.6]   0.40           0.39           
120     140     62.7 [61.5, 63.8]   0.40           0.39           
140     160     61.7 [60.0, 63.3]   0.51           0.50           
160     180     57.7 [55.2, 60.1]   1.12           1.11           
180     200     58.0 [54.7, 

5000000/5000000 [==============================] - 344s 69us/step - loss: 2.0275 - acc: 0.8866 - weighted_acc: 0.8875 - val_loss: 2.2881 - val_acc: 0.8782 - val_weighted_acc: 0.8792

Epoch 00018: val_acc improved from 0.87739 to 0.87818, saving model to 20L1024N_0610_s1_acc.hdf5

Epoch 00018: val_weighted_acc did not improve from 0.87973

Epoch 00018: val_loss did not improve from 2.22811
Estimating new weights...
Average threshold for eff=0.6: 0.8107718467712404
tau_vs_jet fpr at threshold:  0.01981380864653568
pt min  pt max  eff                 old weight     new weight     
20      30      58.8 [58.6, 58.9]   1.38           1.40           
30      40      60.3 [60.1, 60.5]   0.76           0.75           
40      50      60.7 [60.4, 60.9]   0.73           0.72           
50      60      60.1 [59.8, 60.5]   0.82           0.81           
60      70      61.4 [61.0, 61.9]   0.59           0.58           
70      80      62.2 [61.6, 62.8]   0.48           0.46           
80      90   

Average threshold for eff=0.6: 0.6299660325050356
tau_vs_jet fpr at threshold:  0.022375118737265647
pt min  pt max  eff                 old weight     new weight     
20      30      60.2 [60.1, 60.4]   1.43           1.43           
30      40      60.1 [59.9, 60.3]   0.72           0.72           
40      50      59.6 [59.3, 59.8]   0.70           0.71           
50      60      58.6 [58.3, 59.0]   0.83           0.85           
60      70      58.5 [58.1, 59.0]   0.55           0.56           
70      80      59.9 [59.3, 60.5]   0.44           0.44           
80      90      58.6 [57.8, 59.3]   0.35           0.36           
90      100     60.1 [59.2, 61.0]   0.39           0.39           
100     120     60.6 [59.8, 61.5]   0.44           0.44           
120     140     65.5 [64.3, 66.7]   0.50           0.46           
140     160     68.8 [67.2, 70.3]   0.65           0.57           
160     180     65.9 [63.5, 68.1]   1.77           1.61           
180     200     70.2 [67.0, 

...

pt min  pt max  eff                 old weight     new weight     
20      30      63.0 [62.8, 63.1]   1.50           1.45           
30      40      59.0 [58.8, 59.2]   0.58           0.60           
40      50      55.9 [55.6, 56.2]   0.79           0.86           
50      60      56.5 [56.1, 56.9]   0.90           0.97           
60      70      56.1 [55.6, 56.5]   0.41           0.44           
70      80      57.6 [57.0, 58.1]   0.52           0.55           
80      90      58.7 [58.0, 59.5]   0.55           0.57           
90      100     60.9 [60.0, 61.8]   0.86           0.86           
100     120     61.6 [60.8, 62.5]   0.41           0.40           
120     140     61.8 [60.6, 63.0]   0.06           0.06           
140     160     60.7 [58.9, 62.4]   0.02           0.02           
160     180     62.9 [60.6, 65.2]   0.24           0.23           
180     200     59.8 [56.5, 63.1]   0.52           0.52           
200     1000    59.4 [57.0, 61.8]   1.23           1.24       

5000000/5000000 [==============================] - 342s 68us/step - loss: 1.7570 - acc: 0.9049 - weighted_acc: 0.9065 - val_loss: 2.3904 - val_acc: 0.8480 - val_weighted_acc: 0.8509

Epoch 00134: val_acc did not improve from 0.90506

Epoch 00134: val_weighted_acc did not improve from 0.90624

Epoch 00134: val_loss did not improve from 1.85273
Estimating new weights...
Average threshold for eff=0.6: 0.6531074643135071
tau_vs_jet fpr at threshold:  0.025098322966144223
pt min  pt max  eff                 old weight     new weight     
20      30      61.8 [61.6, 61.9]   1.47           1.43           
30      40      56.0 [55.9, 56.2]   0.60           0.65           
40      50      58.4 [58.2, 58.7]   0.85           0.88           
50      60      61.6 [61.3, 62.0]   0.88           0.87           
60      70      64.3 [63.8, 64.7]   0.42           0.40           
70      80      64.8 [64.2, 65.3]   0.55           0.52           
80      90      62.9 [62.2, 63.6]   0.57           0.54    

pt min  pt max  eff                 old weight     new weight     
20      30      60.6 [60.5, 60.8]   1.45           1.44           
30      40      59.4 [59.3, 59.6]   0.62           0.63           
40      50      58.5 [58.3, 58.8]   0.86           0.88           
50      60      60.3 [59.9, 60.6]   0.83           0.83           
60      70      61.1 [60.6, 61.5]   0.41           0.40           
70      80      60.0 [59.4, 60.6]   0.58           0.58           
80      90      58.6 [57.9, 59.4]   0.60           0.61           
90      100     60.6 [59.7, 61.4]   0.95           0.95           
100     120     60.3 [59.4, 61.1]   0.38           0.38           
120     140     63.2 [62.0, 64.4]   0.05           0.05           
140     160     60.5 [58.8, 62.1]   0.02           0.02           
160     180     63.2 [60.9, 65.5]   0.28           0.27           
180     200     63.4 [60.2, 66.5]   0.74           0.70           
200     1000    63.6 [61.0, 66.1]   1.38           1.31       

5000000/5000000 [==============================] - 342s 68us/step - loss: 1.7431 - acc: 0.9055 - weighted_acc: 0.9078 - val_loss: 1.9118 - val_acc: 0.9012 - val_weighted_acc: 0.9032

Epoch 00145: val_acc did not improve from 0.90506

Epoch 00145: val_weighted_acc did not improve from 0.90624

Epoch 00145: val_loss did not improve from 1.85273
Estimating new weights...
Average threshold for eff=0.6: 0.7630564093589783
tau_vs_jet fpr at threshold:  0.013451814604693765
pt min  pt max  eff                 old weight     new weight     
20      30      59.2 [59.0, 59.3]   1.50           1.52           
30      40      60.1 [59.9, 60.3]   0.62           0.61           
40      50      61.1 [60.8, 61.3]   0.81           0.80           
50      60      61.1 [60.7, 61.4]   0.74           0.72           
60      70      61.7 [61.2, 62.1]   0.36           0.35           
70      80      60.0 [59.5, 60.6]   0.51           0.51           
80      90      59.4 [58.7, 60.1]   0.52           0.51    

pt min  pt max  eff                 old weight     new weight     
20      30      52.1 [51.9, 52.2]   1.47           1.59           
30      40      60.4 [60.2, 60.6]   0.64           0.60           
40      50      63.7 [63.5, 64.0]   0.83           0.74           
50      60      68.3 [68.0, 68.7]   0.74           0.61           
60      70      73.1 [72.7, 73.5]   0.38           0.29           
70      80      77.8 [77.3, 78.2]   0.58           0.42           
80      90      81.0 [80.4, 81.5]   0.56           0.39           
90      100     80.2 [79.5, 80.9]   0.81           0.57           
100     120     82.2 [81.5, 82.8]   0.30           0.20           
120     140     81.9 [80.9, 82.8]   0.04           0.03           
140     160     83.0 [81.7, 84.3]   0.01           0.01           
160     180     77.6 [75.5, 79.6]   0.27           0.20           
180     200     76.9 [73.9, 79.6]   0.70           0.51           
200     1000    79.5 [77.4, 81.5]   1.31           0.92       

In [22]:
model_name = '20L1024N_0610'
model = load_model('20L1024N_0610_s1_loss.hdf5', custom_objects = {'tau_crossentropy': tau_crossentropy})
opt = keras.optimizers.Adam(lr=1e-4)
#model.compile(loss="categorical_crossentropy", optimizer=opt, weighted_metrics=["accuracy"])
model.compile(loss=tau_crossentropy, optimizer=opt, weighted_metrics=["accuracy"], metrics=["accuracy"])

In [20]:
bin_weights = np.array([1.50, 0.54, 0.80, 1.02, 0.41, 0.50, 0.54, 0.87, 0.45, 0.07, 0.02, 0.24, 0.51, 1.14])
SetBinWeights(weight_df, bin_weights)
sf, sf_other = UpdateClassWeights(weight_df, tau_class_weight)
bin_weights *= sf
print(sf, sf_other)

1.0006485562877812 1.0


In [25]:
train_name = '{}_0610_s2'.format(model_name)
n_train = int(3e7)
n_test = X.shape[0] - n_train
n_test_weight = int(1e6)
n_per_step = int(5e6)
epoch = 150
n_pass = 25

cb_acc = ModelCheckpoint("{}_acc.hdf5".format(train_name), monitor="val_acc", save_best_only=True,
                         save_weights_only=False, mode="max", verbose=1)
cb_w_acc = ModelCheckpoint("{}_w_acc.hdf5".format(train_name), monitor="val_weighted_acc", save_best_only=True,
                           save_weights_only=False, mode="max", verbose=1)
cb_loss = ModelCheckpoint("{}_loss.hdf5".format(train_name), monitor="val_loss", save_best_only=True,
                          save_weights_only=False, mode="min", verbose=1)
log_name = "{}.log".format(train_name)
if os.path.isfile(log_name):
    os.remove(log_name)
csv_log = CSVLogger(log_name, append=True)

for pass_id in range(0, n_pass):
    print("Starting pass {}".format(pass_id))
    for pos in range(0, n_train, n_per_step):
        if epoch >= 5:
            val_data = (X[n_train:n_train+n_test, :], Y[n_train:n_train+n_test, :],
                                          weight_df.weight[n_train:n_train+n_test].values)
            callbacks = [cb_acc, cb_w_acc, cb_loss, csv_log]
        else:
            val_data = None
            callbacks = [csv_log]
        fit_hist = model.fit(X[pos:pos+n_per_step, :], Y[pos:pos+n_per_step, :], callbacks=callbacks,
                             validation_data=val_data, batch_size=10000, epochs=epoch+1, initial_epoch=epoch, verbose=1,
                             sample_weight=weight_df.weight[pos:pos+n_per_step].values)
        print("Estimating new weights...")
        #boost = 1 + 0.1 / (epoch + 1)
        bin_weights = UpdateWeights(model, weight_df, bin_weights, X, pos, n_test_weight, 0.6,
                                    boost=1, tau_class_weight=tau_class_weight)
        epoch += 1
model.save("{}_final.hdf5".format(train_name))

Starting pass 0
Train on 5000000 samples, validate on 5000000 samples
Epoch 151/151
5000000/5000000 [==============================] - 348s 70us/step - loss: 1.8301 - acc: 0.9068 - weighted_acc: 0.9088 - val_loss: 1.9131 - val_acc: 0.9020 - val_weighted_acc: 0.9054

Epoch 00151: val_acc improved from -inf to 0.90196, saving model to 20L1024N_0610_0610_s2_acc.hdf5

Epoch 00151: val_weighted_acc improved from -inf to 0.90538, saving model to 20L1024N_0610_0610_s2_w_acc.hdf5

Epoch 00151: val_loss improved from inf to 1.91310, saving model to 20L1024N_0610_0610_s2_loss.hdf5
Estimating new weights...
Average threshold for eff=0.6: 0.8576518058776856
tau_vs_jet fpr at threshold:  0.013650425728785996
pt min  pt max  eff                 old weight     new weight     
20      30      62.5 [62.4, 62.6]   1.50           1.46           
30      40      56.4 [56.2, 56.6]   0.54           0.58           
40      50      58.2 [58.0, 58.5]   0.80           0.84           
50      60      61.1 [60.8,

5000000/5000000 [==============================] - 345s 69us/step - loss: 1.8092 - acc: 0.9084 - weighted_acc: 0.9100 - val_loss: 1.8620 - val_acc: 0.9041 - val_weighted_acc: 0.9061

Epoch 00156: val_acc did not improve from 0.90938

Epoch 00156: val_weighted_acc did not improve from 0.91104

Epoch 00156: val_loss did not improve from 1.81853
Estimating new weights...
Average threshold for eff=0.6: 0.8911638498306275
tau_vs_jet fpr at threshold:  0.01172930713624419
pt min  pt max  eff                 old weight     new weight     
20      30      59.1 [59.0, 59.3]   1.40           1.42           
30      40      59.2 [59.0, 59.3]   0.72           0.72           
40      50      61.8 [61.6, 62.1]   0.87           0.84           
50      60      63.6 [63.2, 63.9]   0.88           0.83           
60      70      60.8 [60.3, 61.2]   0.36           0.35           
70      80      60.6 [60.0, 61.2]   0.49           0.48           
80      90      62.5 [61.8, 63.2]   0.50           0.48     


Epoch 00161: val_loss improved from 1.81316 to 1.77301, saving model to 20L1024N_0610_0610_s2_loss.hdf5
Estimating new weights...
Average threshold for eff=0.6: 0.901858925819397
tau_vs_jet fpr at threshold:  0.01137777238387087
pt min  pt max  eff                 old weight     new weight     
20      30      58.2 [58.0, 58.3]   1.42           1.45           
30      40      60.9 [60.8, 61.1]   0.78           0.77           
40      50      62.8 [62.5, 63.0]   0.79           0.74           
50      60      62.7 [62.4, 63.0]   0.69           0.66           
60      70      59.4 [59.0, 59.9]   0.34           0.34           
70      80      59.6 [59.0, 60.2]   0.50           0.49           
80      90      59.9 [59.2, 60.7]   0.46           0.45           
90      100     59.6 [58.7, 60.5]   0.74           0.74           
100     120     61.0 [60.1, 61.8]   0.35           0.34           
120     140     61.2 [60.0, 62.4]   0.06           0.06           
140     160     60.8 [59.1, 62.4]

5000000/5000000 [==============================] - 344s 69us/step - loss: 1.7955 - acc: 0.9083 - weighted_acc: 0.9106 - val_loss: 1.7763 - val_acc: 0.9109 - val_weighted_acc: 0.9132

Epoch 00167: val_acc did not improve from 0.91110

Epoch 00167: val_weighted_acc improved from 0.91284 to 0.91317, saving model to 20L1024N_0610_0610_s2_w_acc.hdf5

Epoch 00167: val_loss did not improve from 1.77301
Estimating new weights...
Average threshold for eff=0.6: 0.8803452253341675
tau_vs_jet fpr at threshold:  0.011195728025728937
pt min  pt max  eff                 old weight     new weight     
20      30      58.7 [58.6, 58.8]   1.46           1.49           
30      40      60.8 [60.6, 60.9]   0.78           0.77           
40      50      61.9 [61.7, 62.2]   0.68           0.65           
50      60      60.6 [60.3, 61.0]   0.59           0.58           
60      70      58.9 [58.5, 59.4]   0.37           0.37           
70      80      60.1 [59.6, 60.7]   0.55           0.54           
80   

Average threshold for eff=0.6: 0.9168730854988099
tau_vs_jet fpr at threshold:  0.01170601727086702
pt min  pt max  eff                 old weight     new weight     
20      30      59.8 [59.7, 59.9]   1.47           1.47           
30      40      60.7 [60.5, 60.9]   0.77           0.76           
40      50      60.6 [60.4, 60.9]   0.64           0.64           
50      60      60.0 [59.7, 60.4]   0.60           0.60           
60      70      58.2 [57.7, 58.6]   0.43           0.44           
70      80      58.6 [58.0, 59.2]   0.60           0.62           
80      90      58.8 [58.1, 59.5]   0.47           0.48           
90      100     60.1 [59.2, 61.0]   0.78           0.78           
100     120     58.9 [58.0, 59.7]   0.31           0.32           
120     140     56.1 [54.9, 57.3]   0.05           0.05           
140     160     57.6 [55.9, 59.3]   0.02           0.02           
160     180     51.8 [49.4, 54.1]   0.31           0.36           
180     200     58.0 [54.8, 6

...

5000000/5000000 [==============================] - 351s 70us/step - loss: 1.7640 - acc: 0.9105 - weighted_acc: 0.9126 - val_loss: 1.7473 - val_acc: 0.9124 - val_weighted_acc: 0.9145

Epoch 00277: val_acc did not improve from 0.91309

Epoch 00277: val_weighted_acc did not improve from 0.91534

Epoch 00277: val_loss did not improve from 1.73666
Estimating new weights...
Average threshold for eff=0.6: 0.8850747466087342
tau_vs_jet fpr at threshold:  0.011089510528726178
pt min  pt max  eff                 old weight     new weight     
20      30      59.6 [59.4, 59.7]   1.45           1.46           
30      40      60.5 [60.3, 60.7]   0.80           0.79           
40      50      60.3 [60.0, 60.5]   0.59           0.59           
50      60      60.2 [59.8, 60.5]   0.70           0.70           
60      70      60.7 [60.3, 61.2]   0.46           0.45           
70      80      60.0 [59.4, 60.6]   0.55           0.55           
80      90      58.8 [58.1, 59.5]   0.31           0.32    

pt min  pt max  eff                 old weight     new weight     
20      30      60.2 [60.0, 60.3]   1.47           1.46           
30      40      60.1 [59.9, 60.3]   0.78           0.78           
40      50      60.0 [59.8, 60.3]   0.58           0.59           
50      60      60.0 [59.7, 60.4]   0.71           0.71           
60      70      59.3 [58.8, 59.7]   0.47           0.47           
70      80      59.3 [58.7, 59.9]   0.57           0.58           
80      90      59.9 [59.2, 60.6]   0.32           0.32           
90      100     57.3 [56.5, 58.2]   0.92           0.96           
100     120     59.8 [58.9, 60.6]   0.19           0.19           
120     140     58.1 [56.9, 59.3]   0.04           0.04           
140     160     58.6 [56.9, 60.2]   0.02           0.02           
160     180     57.7 [55.2, 60.1]   2.91           2.91           
180     200     57.3 [53.9, 60.5]   1.23           1.23           
200     1000    56.3 [53.9, 58.8]   0.46           0.49       

5000000/5000000 [==============================] - 345s 69us/step - loss: 1.7613 - acc: 0.9106 - weighted_acc: 0.9127 - val_loss: 1.7775 - val_acc: 0.9084 - val_weighted_acc: 0.9109

Epoch 00288: val_acc did not improve from 0.91309

Epoch 00288: val_weighted_acc did not improve from 0.91534

Epoch 00288: val_loss did not improve from 1.73666
Estimating new weights...
Average threshold for eff=0.6: 0.8776607751846313
tau_vs_jet fpr at threshold:  0.010955458090580083
pt min  pt max  eff                 old weight     new weight     
20      30      60.4 [60.2, 60.5]   1.46           1.46           
30      40      59.8 [59.6, 60.0]   0.77           0.78           
40      50      59.8 [59.6, 60.1]   0.59           0.59           
50      60      60.0 [59.7, 60.4]   0.73           0.73           
60      70      59.1 [58.7, 59.6]   0.47           0.48           
70      80      59.8 [59.2, 60.4]   0.58           0.58           
80      90      60.0 [59.3, 60.7]   0.31           0.31    

pt min  pt max  eff                 old weight     new weight     
20      30      59.8 [59.7, 60.0]   1.42           1.42           
30      40      60.5 [60.3, 60.7]   0.81           0.81           
40      50      60.0 [59.8, 60.3]   0.62           0.62           
50      60      59.8 [59.5, 60.2]   0.76           0.76           
60      70      59.5 [59.0, 60.0]   0.49           0.49           
70      80      59.5 [58.9, 60.0]   0.60           0.60           
80      90      59.3 [58.6, 60.0]   0.32           0.32           
90      100     59.0 [58.1, 59.9]   0.99           1.00           
100     120     59.9 [59.0, 60.7]   0.20           0.20           
120     140     60.7 [59.5, 61.9]   0.04           0.04           
140     160     62.3 [60.6, 63.9]   0.01           0.01           
160     180     57.1 [54.7, 59.5]   2.99           3.14           
180     200     57.4 [54.1, 60.6]   0.86           0.86           
200     1000    65.0 [62.5, 67.3]   0.42           0.38       

5000000/5000000 [==============================] - 345s 69us/step - loss: 1.7594 - acc: 0.9107 - weighted_acc: 0.9128 - val_loss: 1.7431 - val_acc: 0.9119 - val_weighted_acc: 0.9142

Epoch 00299: val_acc did not improve from 0.91343

Epoch 00299: val_weighted_acc did not improve from 0.91556

Epoch 00299: val_loss did not improve from 1.73206
Estimating new weights...
Average threshold for eff=0.6: 0.8997509002685546
tau_vs_jet fpr at threshold:  0.011188726319646554
pt min  pt max  eff                 old weight     new weight     
20      30      59.5 [59.4, 59.6]   1.45           1.46           
30      40      60.9 [60.7, 61.0]   0.79           0.78           
40      50      60.4 [60.2, 60.7]   0.59           0.58           
50      60      59.9 [59.6, 60.3]   0.74           0.73           
60      70      59.7 [59.2, 60.2]   0.47           0.47           
70      80      59.5 [59.0, 60.1]   0.57           0.56           
80      90      59.3 [58.6, 60.0]   0.31           0.30    

In [18]:
model_name = '20L1024N'
model = load_model('20L1024N_0610_s2_loss.hdf5', custom_objects = {'tau_crossentropy': tau_crossentropy})
opt = keras.optimizers.Adam(lr=1e-5)
#model.compile(loss="categorical_crossentropy", optimizer=opt, weighted_metrics=["accuracy"])
model.compile(loss=tau_crossentropy, optimizer=opt, weighted_metrics=["accuracy", tau_e_loss, tau_mu_loss, tau_jet_loss],
              metrics=["accuracy", tau_e_loss, tau_mu_loss, tau_jet_loss])

In [19]:
bin_weights = np.array([1.44, 0.80, 0.60, 0.73, 0.48, 0.58, 0.31, 0.99, 0.19, 0.04, 0.01, 2.83, 0.76, 0.40])
SetBinWeights(weight_df, bin_weights)
sf, sf_other = UpdateClassWeights(weight_df, tau_class_weight)
bin_weights *= sf
print(sf, sf_other)

1.0008504996460246 1.0000000000000002


In [20]:
train_name = '{}_0610_s3'.format(model_name)
n_train = int(3e7)
n_test = X.shape[0] - n_train
n_test_weight = int(1e6)
n_per_step = int(5e6)
epoch = 300
n_pass = 25

cb_acc = ModelCheckpoint("{}_acc.hdf5".format(train_name), monitor="val_acc", save_best_only=True,
                         save_weights_only=False, mode="max", verbose=1)
cb_w_acc = ModelCheckpoint("{}_w_acc.hdf5".format(train_name), monitor="val_weighted_acc", save_best_only=True,
                           save_weights_only=False, mode="max", verbose=1)
cb_loss = ModelCheckpoint("{}_loss.hdf5".format(train_name), monitor="val_loss", save_best_only=True,
                          save_weights_only=False, mode="min", verbose=1)
log_name = "{}.0.log".format(train_name)
if os.path.isfile(log_name):
    os.remove(log_name)
csv_log = CSVLogger(log_name, append=True)

for pass_id in range(0, n_pass):
    print("Starting pass {}".format(pass_id))
    for pos in range(0, n_train, n_per_step):
        if epoch >= 5:
            val_data = (X[n_train:n_train+n_test, :], Y[n_train:n_train+n_test, :],
                                          weight_df.weight[n_train:n_train+n_test].values)
            callbacks = [cb_acc, cb_w_acc, cb_loss, csv_log]
        else:
            val_data = None
            callbacks = [csv_log]
        fit_hist = model.fit(X[pos:pos+n_per_step, :], Y[pos:pos+n_per_step, :], callbacks=callbacks,
                             validation_data=val_data, batch_size=10000, epochs=epoch+1, initial_epoch=epoch, verbose=1,
                             sample_weight=weight_df.weight[pos:pos+n_per_step].values)
        print("Estimating new weights...")
        #boost = 1 + 0.1 / (epoch + 1)
        bin_weights = UpdateWeights(model, weight_df, bin_weights, X, pos, n_test_weight, 0.6,
                                    boost=1, tau_class_weight=tau_class_weight)
        epoch += 1
model.save("{}_final.hdf5".format(train_name))

Starting pass 0
Train on 5000000 samples, validate on 5000000 samples
Epoch 301/301
5000000/5000000 [==============================] - 349s 70us/step - loss: 0.3914 - acc: 0.9110 - tau_e_loss: 0.1489 - tau_mu_loss: 0.1018 - tau_jet_loss: 0.1640 - weighted_acc: 0.9130 - weighted_tau_e_loss: 0.1358 - weighted_tau_mu_loss: 0.0937 - weighted_tau_jet_loss: 0.1619 - val_loss: 0.3984 - val_acc: 0.9077 - val_tau_e_loss: 0.1767 - val_tau_mu_loss: 0.0793 - val_tau_jet_loss: 0.1628 - val_weighted_acc: 0.9102 - val_weighted_tau_e_loss: 0.1638 - val_weighted_tau_mu_loss: 0.0742 - val_weighted_tau_jet_loss: 0.1603

Epoch 00301: val_acc improved from -inf to 0.90768, saving model to 20L1024N_0610_s3_acc.hdf5

Epoch 00301: val_weighted_acc improved from -inf to 0.91018, saving model to 20L1024N_0610_s3_w_acc.hdf5

Epoch 00301: val_loss improved from inf to 0.39837, saving model to 20L1024N_0610_s3_loss.hdf5
Estimating new weights...
Average threshold for eff=0.6: 0.8914531469345093
tau_vs_jet fpr at t


Epoch 00305: val_acc did not improve from 0.91238

Epoch 00305: val_weighted_acc did not improve from 0.91448

Epoch 00305: val_loss improved from 0.37629 to 0.37315, saving model to 20L1024N_0610_s3_loss.hdf5
Estimating new weights...
Average threshold for eff=0.6: 0.9035053968429565
tau_vs_jet fpr at threshold:  0.010833973211472529
pt min  pt max  eff                 old weight     new weight     
20      30      59.6 [59.5, 59.8]   1.43           1.43           
30      40      60.6 [60.4, 60.8]   0.82           0.81           
40      50      60.3 [60.0, 60.5]   0.60           0.60           
50      60      60.0 [59.7, 60.4]   0.74           0.74           
60      70      59.9 [59.4, 60.4]   0.48           0.48           
70      80      59.6 [59.1, 60.2]   0.58           0.58           
80      90      59.6 [58.9, 60.3]   0.30           0.30           
90      100     58.3 [57.4, 59.2]   0.97           0.99           
100     120     59.8 [58.9, 60.6]   0.20           0.20    

5000000/5000000 [==============================] - 343s 69us/step - loss: 0.3850 - acc: 0.9110 - tau_e_loss: 0.1485 - tau_mu_loss: 0.0937 - tau_jet_loss: 0.1643 - weighted_acc: 0.9128 - weighted_tau_e_loss: 0.1361 - weighted_tau_mu_loss: 0.0867 - weighted_tau_jet_loss: 0.1622 - val_loss: 0.3639 - val_acc: 0.9137 - val_tau_e_loss: 0.1487 - val_tau_mu_loss: 0.0732 - val_tau_jet_loss: 0.1600 - val_weighted_acc: 0.9156 - val_weighted_tau_e_loss: 0.1370 - val_weighted_tau_mu_loss: 0.0688 - val_weighted_tau_jet_loss: 0.1581

Epoch 00310: val_acc improved from 0.91238 to 0.91367, saving model to 20L1024N_0610_s3_acc.hdf5

Epoch 00310: val_weighted_acc improved from 0.91448 to 0.91565, saving model to 20L1024N_0610_s3_w_acc.hdf5

Epoch 00310: val_loss improved from 0.37315 to 0.36390, saving model to 20L1024N_0610_s3_loss.hdf5
Estimating new weights...
Average threshold for eff=0.6: 0.9079348683357239
tau_vs_jet fpr at threshold:  0.011006036170356734
pt min  pt max  eff                 old we


Epoch 00314: val_acc did not improve from 0.91367

Epoch 00314: val_weighted_acc did not improve from 0.91565

Epoch 00314: val_loss did not improve from 0.36390
Estimating new weights...
Average threshold for eff=0.6: 0.8955326557159424
tau_vs_jet fpr at threshold:  0.011313268056998144
pt min  pt max  eff                 old weight     new weight     
20      30      59.8 [59.6, 59.9]   1.44           1.44           
30      40      60.2 [60.0, 60.3]   0.80           0.80           
40      50      60.1 [59.9, 60.4]   0.59           0.59           
50      60      60.4 [60.0, 60.8]   0.74           0.73           
60      70      59.7 [59.2, 60.2]   0.48           0.48           
70      80      60.4 [59.8, 61.0]   0.57           0.57           
80      90      60.9 [60.1, 61.6]   0.29           0.29           
90      100     60.7 [59.8, 61.5]   1.05           1.05           
100     120     59.8 [59.0, 60.7]   0.21           0.20           
120     140     60.1 [58.9, 61.3]   0.05

5000000/5000000 [==============================] - 343s 69us/step - loss: 0.3830 - acc: 0.9114 - tau_e_loss: 0.1481 - tau_mu_loss: 0.0908 - tau_jet_loss: 0.1639 - weighted_acc: 0.9132 - weighted_tau_e_loss: 0.1363 - weighted_tau_mu_loss: 0.0846 - weighted_tau_jet_loss: 0.1622 - val_loss: 0.3813 - val_acc: 0.9107 - val_tau_e_loss: 0.1636 - val_tau_mu_loss: 0.0744 - val_tau_jet_loss: 0.1611 - val_weighted_acc: 0.9129 - val_weighted_tau_e_loss: 0.1519 - val_weighted_tau_mu_loss: 0.0703 - val_weighted_tau_jet_loss: 0.1590

Epoch 00319: val_acc did not improve from 0.91367

Epoch 00319: val_weighted_acc did not improve from 0.91565

Epoch 00319: val_loss did not improve from 0.36390
Estimating new weights...
Average threshold for eff=0.6: 0.905507481098175
tau_vs_jet fpr at threshold:  0.010792762143317787
pt min  pt max  eff                 old weight     new weight     
20      30      59.7 [59.6, 59.9]   1.44           1.45           
30      40      60.1 [59.9, 60.3]   0.79           0.

...

Average threshold for eff=0.6: 0.896724510192871
tau_vs_jet fpr at threshold:  0.010740617130374101
pt min  pt max  eff                 old weight     new weight     
20      30      59.3 [59.2, 59.5]   1.44           1.45           
30      40      60.5 [60.3, 60.7]   0.78           0.77           
40      50      60.5 [60.2, 60.7]   0.56           0.56           
50      60      60.3 [59.9, 60.6]   0.85           0.84           
60      70      60.5 [60.0, 60.9]   0.42           0.41           
70      80      60.5 [59.9, 61.0]   0.54           0.54           
80      90      60.4 [59.7, 61.1]   0.14           0.14           
90      100     59.5 [58.6, 60.4]   1.59           1.58           
100     120     61.1 [60.2, 61.9]   0.15           0.15           
120     140     63.2 [62.0, 64.4]   0.05           0.04           
140     160     63.8 [62.1, 65.4]   0.00           0.00           
160     180     57.9 [55.5, 60.3]   4.62           4.60           
180     200     58.1 [54.8, 6

5000000/5000000 [==============================] - 343s 69us/step - loss: 0.3754 - acc: 0.9113 - tau_e_loss: 0.1467 - tau_mu_loss: 0.0841 - tau_jet_loss: 0.1649 - weighted_acc: 0.9131 - weighted_tau_e_loss: 0.1347 - weighted_tau_mu_loss: 0.0779 - weighted_tau_jet_loss: 0.1628 - val_loss: 0.3559 - val_acc: 0.9138 - val_tau_e_loss: 0.1457 - val_tau_mu_loss: 0.0677 - val_tau_jet_loss: 0.1594 - val_weighted_acc: 0.9157 - val_weighted_tau_e_loss: 0.1349 - val_weighted_tau_mu_loss: 0.0635 - val_weighted_tau_jet_loss: 0.1576

Epoch 00436: val_acc did not improve from 0.91421

Epoch 00436: val_weighted_acc did not improve from 0.91611

Epoch 00436: val_loss improved from 0.35760 to 0.35595, saving model to 20L1024N_0610_s3_loss.hdf5
Estimating new weights...
Average threshold for eff=0.6: 0.9033067226409912
tau_vs_jet fpr at threshold:  0.010782042218193443
pt min  pt max  eff                 old weight     new weight     
20      30      59.5 [59.4, 59.7]   1.44           1.45           
30  

Average threshold for eff=0.6: 0.8967133522033692
tau_vs_jet fpr at threshold:  0.010902470619813974
pt min  pt max  eff                 old weight     new weight     
20      30      59.7 [59.6, 59.9]   1.45           1.46           
30      40      60.0 [59.8, 60.2]   0.77           0.77           
40      50      60.0 [59.7, 60.3]   0.56           0.56           
50      60      60.5 [60.1, 60.8]   0.85           0.84           
60      70      60.0 [59.5, 60.4]   0.41           0.41           
70      80      60.5 [60.0, 61.1]   0.54           0.54           
80      90      61.0 [60.3, 61.8]   0.13           0.13           
90      100     61.3 [60.4, 62.2]   1.65           1.61           
100     120     61.1 [60.3, 62.0]   0.15           0.14           
120     140     61.9 [60.7, 63.1]   0.04           0.04           
140     160     60.4 [58.7, 62.1]   0.00           0.00           
160     180     63.3 [61.0, 65.6]   4.67           4.42           
180     200     63.1 [59.8, 

5000000/5000000 [==============================] - 345s 69us/step - loss: 0.3723 - acc: 0.9116 - tau_e_loss: 0.1464 - tau_mu_loss: 0.0810 - tau_jet_loss: 0.1644 - weighted_acc: 0.9134 - weighted_tau_e_loss: 0.1348 - weighted_tau_mu_loss: 0.0749 - weighted_tau_jet_loss: 0.1626 - val_loss: 0.4190 - val_acc: 0.9036 - val_tau_e_loss: 0.1918 - val_tau_mu_loss: 0.0777 - val_tau_jet_loss: 0.1665 - val_weighted_acc: 0.9059 - val_weighted_tau_e_loss: 0.1799 - val_weighted_tau_mu_loss: 0.0747 - val_weighted_tau_jet_loss: 0.1645

Epoch 00445: val_acc did not improve from 0.91421

Epoch 00445: val_weighted_acc did not improve from 0.91611

Epoch 00445: val_loss did not improve from 0.35595
Estimating new weights...
Average threshold for eff=0.6: 0.8906076908111572
tau_vs_jet fpr at threshold:  0.011297468058658043
pt min  pt max  eff                 old weight     new weight     
20      30      60.2 [60.0, 60.3]   1.45           1.44           
30      40      59.8 [59.6, 60.0]   0.77           0

Average threshold for eff=0.6: 0.9000776648521422
tau_vs_jet fpr at threshold:  0.01076862395470363
pt min  pt max  eff                 old weight     new weight     
20      30      59.4 [59.3, 59.6]   1.44           1.45           
30      40      60.4 [60.2, 60.6]   0.78           0.77           
40      50      60.3 [60.0, 60.5]   0.56           0.56           
50      60      60.1 [59.8, 60.5]   0.87           0.86           
60      70      60.3 [59.8, 60.8]   0.41           0.40           
70      80      60.4 [59.8, 60.9]   0.53           0.53           
80      90      60.6 [59.9, 61.3]   0.12           0.12           
90      100     59.9 [59.1, 60.8]   1.57           1.57           
100     120     61.3 [60.5, 62.1]   0.14           0.14           
120     140     63.1 [61.9, 64.3]   0.04           0.04           
140     160     63.5 [61.8, 65.1]   0.00           0.00           
160     180     58.4 [56.0, 60.7]   4.25           4.23           
180     200     58.1 [54.8, 6

In [14]:
model_name = '20L1024N'
model = load_model('20L1024N_0610_s3_loss.hdf5', custom_objects = {
    'tau_crossentropy': tau_crossentropy,
    'tau_e_loss': tau_e_loss, 'tau_mu_loss': tau_mu_loss, 'tau_jet_loss': tau_jet_loss})
opt = keras.optimizers.Adam(lr=1e-3)
#model.compile(loss="categorical_crossentropy", optimizer=opt, weighted_metrics=["accuracy"])
model.compile(loss=tau_crossentropy, optimizer=opt, weighted_metrics=["accuracy", tau_e_loss, tau_mu_loss, tau_jet_loss],
              metrics=["accuracy", tau_e_loss, tau_mu_loss, tau_jet_loss])

In [26]:
model.compile(loss=tau_crossentropy, optimizer=opt, weighted_metrics=["accuracy", tau_e_loss, tau_mu_loss, tau_jet_loss],
              metrics=["accuracy", tau_e_loss, tau_mu_loss, tau_jet_loss])

In [22]:
del model
keras.backend.clear_session()
gc.collect()

53290

In [15]:
train_name = '{}_0610_s4'.format(model_name)
n_train = int(3e7)
n_test = X.shape[0] - n_train
n_test_weight = int(1e6)
n_per_step = int(5e6)
epoch = 450
n_pass = 10

cb_acc = ModelCheckpoint("{}_acc.hdf5".format(train_name), monitor="val_acc", save_best_only=True,
                         save_weights_only=False, mode="max", verbose=1)
cb_w_acc = ModelCheckpoint("{}_w_acc.hdf5".format(train_name), monitor="val_weighted_acc", save_best_only=True,
                           save_weights_only=False, mode="max", verbose=1)
cb_loss = ModelCheckpoint("{}_loss.hdf5".format(train_name), monitor="val_loss", save_best_only=True,
                          save_weights_only=False, mode="min", verbose=1)
log_name = "{}.log".format(train_name)
if os.path.isfile(log_name):
    os.remove(log_name)
csv_log = CSVLogger(log_name, append=True)

for pass_id in range(0, n_pass):
    print("Starting pass {}".format(pass_id))
    for pos in range(0, n_train, n_per_step):
        if epoch >= 5:
            val_data = (X[n_train:n_train+n_test, :], Y[n_train:n_train+n_test, :],
                                          weight_df.weight[n_train:n_train+n_test].values)
            callbacks = [cb_acc, cb_w_acc, cb_loss, csv_log]
        else:
            val_data = None
            callbacks = [csv_log]
        fit_hist = model.fit(X[pos:pos+n_per_step, :], Y[pos:pos+n_per_step, :], callbacks=callbacks,
                             validation_data=val_data, batch_size=10000, epochs=epoch+1, initial_epoch=epoch, verbose=1,
                             sample_weight=weight_df.weight[pos:pos+n_per_step].values)
        print("Estimating new weights...")
        boost = 1 + 0.1 / (epoch + 1)
        bin_weights = UpdateWeights(model, weight_df, bin_weights, X, pos, n_test_weight, 0.6,
                                    boost=1, tau_class_weight=tau_class_weight)
        epoch += 1
model.save("{}_final.hdf5".format(train_name))

Starting pass 0
Train on 5000000 samples, validate on 5000000 samples
Epoch 451/451
5000000/5000000 [==============================] - 349s 70us/step - loss: 0.4343 - acc: 0.9050 - tau_e_loss: 0.1614 - tau_mu_loss: 0.0978 - tau_jet_loss: 0.1750 - weighted_acc: 0.9050 - weighted_tau_e_loss: 0.1614 - weighted_tau_mu_loss: 0.0978 - weighted_tau_jet_loss: 0.1750 - val_loss: 0.5518 - val_acc: 0.8881 - val_tau_e_loss: 0.2063 - val_tau_mu_loss: 0.1388 - val_tau_jet_loss: 0.2068 - val_weighted_acc: 0.8881 - val_weighted_tau_e_loss: 0.2063 - val_weighted_tau_mu_loss: 0.1388 - val_weighted_tau_jet_loss: 0.2068

Epoch 00451: val_acc improved from -inf to 0.88811, saving model to 20L1024N_0610_s4_acc.hdf5

Epoch 00451: val_weighted_acc improved from -inf to 0.88811, saving model to 20L1024N_0610_s4_w_acc.hdf5

Epoch 00451: val_loss improved from inf to 0.55180, saving model to 20L1024N_0610_s4_loss.hdf5
Estimating new weights...
Average threshold for eff=0.6: 0.7306206583976746
tau_vs_jet fpr at t

KeyboardInterrupt: 